# News and its replies

In this notebook, we plan to look for tweets gathered from news accounts and its replies. The article (the full body, perhaps just its lead) might bring us of some "context" that disambiguates whether the reply is hate speech or not

In [1]:
%load_ext autoreload
%autoreload 2
from mongoengine import connect
from tweepyrate import create_apps
from hate_collector.models import Tweet

client = connect("hatespeech-news")
db = client["hatespeech-news"]

We look for tweets that are not replies.

In [2]:
screen_names = [t[1:] for t in db.tweet.distinct('query') if t is not None]
screen_names

['LANACION',
 'clarincom',
 'cronica',
 'elmundoes',
 'elpaisuy',
 'infobae',
 'latercera',
 'lavanguardia',
 'perfilcom']

In [3]:
tweets = Tweet.objects(in_reply_to_status_id=None, user__screen_name__in=screen_names)

print(f"There are {tweets.count() / 1e3:.2f}K that are not replies and come from the official news accounts = {['@' + name for name in screen_names]}")

There are 3.58K that are not replies and come from the official news accounts = ['@LANACION', '@clarincom', '@cronica', '@elmundoes', '@elpaisuy', '@infobae', '@latercera', '@lavanguardia', '@perfilcom']


In [4]:
for _, tw in zip(range(50), tweets):
    print("="*80)
    print(tw.user["screen_name"])
    print(tw.text)

clarincom
Malena Galmarini: "Si le toca, Sergio Massa puede ser uno de los mejores presidentes de la historia" https://t.co/sZxrJZummo
clarincom
Un sector del radicalismo pedirá la expulsión de Ricardo Alfonsín de la UCR por aceptar ser embajador en España https://t.co/7xMdUwdxzu
clarincom
Hernán Lacunza cuestionó la estrategia de Axel Kicillof para negociar la deuda: "Generó incertidumbre innecesaria" https://t.co/7PqMyuIezP
clarincom
El parecido entre Alberto Fernández y Mauricio Macri que nadie quiere ver https://t.co/evMsst3iQR
clarincom
Aplausos y ovación: así recibieron a Ricardo Echegaray en la Aduana tras la suspensión que levantó la AFIP https://t.co/4R92OxbDrY
infobae
🔴 AHORA | Alberto Fernández: “Voy a mandar una ley que termine con la penalización del aborto y permita la atención en cualquier centro público” https://t.co/FgA5huyQQg https://t.co/wppoRRfte6
elmundoes
La Fiscalía pide 5 años de cárcel para un independentista que empujó por las escaleras del metro a un hombre p

## News and its replies

In [5]:
tweets_and_replies = db.tweet.aggregate([ 

    {
        "$match": {
            "user.screen_name": {"$in": screen_names },
            "in_reply_to_status_id": None
        }
    },
    { 
        "$lookup": {   
            "from": "tweet",   
            "localField": "_id",   
            "foreignField": "in_reply_to_status_id", 
            "as": "replies"
        }
    },
    {
        "$match": {
            "$expr": {"$gt": [{"$size": "$replies"}, 20]},
            #"replies": { "$gt": {"$size": 5 } },
        }
    }
])


In [6]:
count = 0

for i, tweet in enumerate(tweets_and_replies):
    
    if len(tweet["replies"]) < 5:
        continue
    
    count += 1
    print(f"{'*' * 80} \n" * 4)
    print(tweet["created_at"])
    print(f"Tweet nro {count}")
    
    print(f"@{tweet['user']['screen_name']}")
    print(tweet["text"])
    
    print("\nRespuestas:")
    
    for reply in tweet["replies"]:
        print(f"- {reply['text']}")

******************************************************************************** 
******************************************************************************** 
******************************************************************************** 
******************************************************************************** 

2020-02-04 10:02:31
Tweet nro 1
@clarincom
Malena Galmarini: "Si le toca, Sergio Massa puede ser uno de los mejores presidentes de la historia" https://t.co/sZxrJZummo

Respuestas:
- @clarincom Contáte otro Malena!!! https://t.co/VK979svS2i
- @clarincom Asi como anda SERGIO ultimamente... que cambia mucho de pensar de un momento a otro... "si le toca".. podría ser con otra primera dama... Mantendríamos lo dicho de ser asi???
- @clarincom Que mitomana inmoral corrupta Peroncha acomodada en este Payaso inmoral Gobierno Nacional Prroncho
- @clarincom Ya compraste la balanza para pesarla?
- @clarincom Tu marido es un impresentable, que sentado en el sillòn de diputad